In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
from text_loader import TextLoader 

In [3]:
num_layers  = 3
hidden_size = 512
batch_size  = 1
max_length  = 1000

loader = TextLoader("data/hamlet.txt")
vocab_size = len(loader.vocab)

In [4]:
X = tf.placeholder(tf.int32, [None, None]) # [배치, seq_len]
x_one_hot = tf.one_hot(X, vocab_size)   # [배치, seq_len, vocab_size]

cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

initial_state = cells.zero_state(batch_size, tf.float32)
outputs, states = tf.nn.dynamic_rnn(cells, x_one_hot, 
                                    initial_state=initial_state, dtype=tf.float32)

outputs = tf.reshape(outputs, [-1, hidden_size])
logits = layers.fully_connected(outputs, vocab_size,
                                activation_fn=None)
y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

In [6]:
sentence = list()
# 시작 글자 생성
sentence += loader.X[:10].tolist()
print("Start with:", "".join([loader.words[char] for char in sentence]))

saver = tf.train.Saver()
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "checkpoints/char-rnn_35000")
    
    # 매 이터레이션마다 글자 하나씩 생성
    pred_char, state = sess.run([pred, states], feed_dict={X:[sentence[:]]})
    for i in range(1000):
        pred_char, state = sess.run([pred, states], 
            feed_dict={X:[[sentence[-1]]], initial_state: state}) # 이전 스텝에 갖고 있는 state 값을 다음 스텝에 넣어줌
        sentence.append(pred_char[0][-1])
        
sentence = [loader.words[char] for char in sentence]
print("".join(sentence))

Start with: <START>	HAMLET


INFO:tensorflow:Restoring parameters from checkpoints/char-rnn_35000
<START>	HAMLET

ACT I



SCENE III	A room in Polonius' house.


	[Enter LAERTES and OPHELIA]

LAERTES	My necessaries are embark'd: farewell:
	And, sister, as the winds give benefit
	And convoy is assistant, do not sleep,
	But let me hear from you.

OPHELIA	Do you doubt that?

LAERTES	For Hamlet and the trifling of his favour,
	Hold it a fashion and a toy in blood,
	A violet in the youth of primy nature,
	Forward, not permanent, sweet, not lasting,
	The perfume and suppliance of a minute; No more.

OPHELIA	       No more but so?

LAERTES	Think it no more;
	For nature, crescent, does not grow alone
	In thews and bulk, but, as this temple waxes,
	The inward service of the mind and soul
	Grows wide withal. Perhaps he loves you now,
	And now no soil nor cautel doth besmirch
	The virtue of his will: but you must fear,
	His greatness weigh'd, his will is not his own;
	For he himself is subject to